## Lab 10 : R y Python


### @autor: Roberto Mendoza

In [1]:
# import libraries 

# For data wrangling 

import pandas as pd 
import numpy as np
import re 
from tqdm import tqdm  # controlar el tiempo en un loop
import os

# Econoemtrics model

import statsmodels.api as sm  # linear regression utiliza todas las columnas de base de datos 
from statsmodels.formula.api import logit, probit # logit and probit
import statsmodels.formula.api as smf  # linear regression usando formula como en R
from linearmodels.iv import IV2SLS # IV regression
from stargazer.stargazer import Stargazer
from sklearn import datasets, linear_model # models 
from sklearn.metrics import mean_squared_error, r2_score
import econtools.metrics as mt  # for econoemtrics models 


# Export latex output

from pystout import pystout  # for export regression table
from statsmodels.iolib.summary2 import summary_col # for export table


import warnings
warnings.filterwarnings('ignore') # eliminar warning messages 

# Si una librería no corre, ya saben instalarlo con pip install 

# !pip install stargazer
# !pip install linearmodels

user = os.getlogin()   # Username
os.chdir(f"C:/Users/{user}/Documents/GitHub/1ECO35_2022_2/Lab10") # Set directorio


## Statsmodel

https://www.statsmodels.org/stable/api.html

In [2]:
# laod dataset

repdata = pd.read_stata(r"../data/dataverse_files/mss_repdata.dta",
                           convert_categoricals=False)

# convert_categoricals=False: No se lee las etiquetas de valores

repdata

,ccode,year,country_name,country_code,GPCP,GPCP_l,GPCP_l2,GPCP_g,GPCP_g_l,GPCP_g_fl,...,muni,state,author,stconst,fh_civ,fh_pol,S,W,WoverS,soc
0,540.0,1981-01-01,Angola,AGO,839.215759,911.847290,1021.776855,-0.079653,-0.107587,0.155680,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
1,540.0,1982-01-01,Angola,AGO,969.864563,839.215759,911.847290,0.155680,-0.079653,-0.034482,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
2,540.0,1983-01-01,Angola,AGO,936.421631,969.864563,839.215759,-0.034482,0.155680,0.059925,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
3,540.0,1984-01-01,Angola,AGO,992.536255,936.421631,969.864563,0.059925,-0.034482,-0.018277,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
4,540.0,1985-01-01,Angola,AGO,974.396118,992.536255,936.421631,-0.018277,0.059925,0.216019,...,0.0,1.0,NaN,NaN,0.000000,0.000000,1.0,0.5,0.500712,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,552.0,1995-01-01,Zimbabwe,ZWE,493.448456,465.000092,680.722412,0.061179,-0.316902,0.513642,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0
739,552.0,1996-01-01,Zimbabwe,ZWE,746.904236,493.448456,465.000092,0.513642,0.061179,0.044380,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0
740,552.0,1997-01-01,Zimbabwe,ZWE,780.051941,746.904236,493.448456,0.044380,0.513642,-0.181482,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0
741,552.0,1998-01-01,Zimbabwe,ZWE,638.486389,780.051941,746.904236,-0.181482,0.044380,0.094420,...,NaN,0.0,NaN,NaN,0.333333,0.333333,1.0,0.5,0.500712,0.0


In [3]:
# summmary statistics

repdata.describe()

# Tipo de variables

repdata.info()

repdata.dtypes

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 0 to 742
Columns: 200 entries, ccode to soc
dtypes: datetime64[ns](1), float32(104), float64(78), int32(9), int8(5), object(3)
memory usage: 813.4+ KB


ccode                  float64
year            datetime64[ns]
country_name            object
country_code            object
GPCP                   float32
                     ...      
fh_pol                 float64
S                      float32
W                      float32
WoverS                 float32
soc                    float32
Length: 200, dtype: object

In [4]:
# Extraer year
# con .month se puede extraer el mes 
# con .day se puede extraer el día 

repdata['time_year'] = pd.DatetimeIndex(repdata['year']).year - 1978
repdata['time_year']

0       3
1       4
2       5
3       6
4       7
       ..
738    17
739    18
740    19
741    20
742    21
Name: time_year, Length: 743, dtype: int64

In [5]:
dummys = pd.get_dummies(repdata["ccode"].astype(int), prefix = "ccode", dummy_na=False)
dummys.columns

# se convierte de float a entera repdata["ccode"].astype(int)
# dummy_na: se omite la dummy para valores missing
# prefix = "ccode": el nombre de estas dummies tienen prefijo ccode

Index(['ccode_404', 'ccode_420', 'ccode_432', 'ccode_433', 'ccode_434',
       'ccode_435', 'ccode_436', 'ccode_437', 'ccode_438', 'ccode_439',
       'ccode_450', 'ccode_451', 'ccode_452', 'ccode_461', 'ccode_471',
       'ccode_475', 'ccode_481', 'ccode_482', 'ccode_483', 'ccode_484',
       'ccode_490', 'ccode_500', 'ccode_501', 'ccode_510', 'ccode_516',
       'ccode_517', 'ccode_520', 'ccode_522', 'ccode_530', 'ccode_540',
       'ccode_541', 'ccode_551', 'ccode_552', 'ccode_553', 'ccode_560',
       'ccode_565', 'ccode_570', 'ccode_571', 'ccode_572', 'ccode_580',
       'ccode_625'],
      dtype='object')

In [6]:
len(dummys.columns)

41

##### Reference get_dummies

https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html

In [7]:
# Dummies a nivel país 

repdata = pd.concat([ repdata , dummys], axis = 1 )

# concatenamos de manera horizontal axis = 1

In [8]:
repdata

,ccode,year,country_name,country_code,GPCP,GPCP_l,GPCP_l2,GPCP_g,GPCP_g_l,GPCP_g_fl,...,ccode_551,ccode_552,ccode_553,ccode_560,ccode_565,ccode_570,ccode_571,ccode_572,ccode_580,ccode_625
0,540.0,1981-01-01,Angola,AGO,839.215759,911.847290,1021.776855,-0.079653,-0.107587,0.155680,...,0,0,0,0,0,0,0,0,0,0
1,540.0,1982-01-01,Angola,AGO,969.864563,839.215759,911.847290,0.155680,-0.079653,-0.034482,...,0,0,0,0,0,0,0,0,0,0
2,540.0,1983-01-01,Angola,AGO,936.421631,969.864563,839.215759,-0.034482,0.155680,0.059925,...,0,0,0,0,0,0,0,0,0,0
3,540.0,1984-01-01,Angola,AGO,992.536255,936.421631,969.864563,0.059925,-0.034482,-0.018277,...,0,0,0,0,0,0,0,0,0,0
4,540.0,1985-01-01,Angola,AGO,974.396118,992.536255,936.421631,-0.018277,0.059925,0.216019,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,552.0,1995-01-01,Zimbabwe,ZWE,493.448456,465.000092,680.722412,0.061179,-0.316902,0.513642,...,0,1,0,0,0,0,0,0,0,0
739,552.0,1996-01-01,Zimbabwe,ZWE,746.904236,493.448456,465.000092,0.513642,0.061179,0.044380,...,0,1,0,0,0,0,0,0,0,0
740,552.0,1997-01-01,Zimbabwe,ZWE,780.051941,746.904236,493.448456,0.044380,0.513642,-0.181482,...,0,1,0,0,0,0,0,0,0,0
741,552.0,1998-01-01,Zimbabwe,ZWE,638.486389,780.051941,746.904236,-0.181482,0.044380,0.094420,...,0,1,0,0,0,0,0,0,0,0


In [9]:
# Creación del trend_country effects

i = 0

while i < 41:  # 41 por el tema de indexing pues en python la posición inicial es cero. 
    var = dummys.columns[i]+"_"+"time"  # creamos el nombre de cada variable
    repdata[var]  = repdata[dummys.columns[i]]*repdata["time_year"] # multiplicacón de variables

    i = i + 1

In [10]:
repdata

,ccode,year,country_name,country_code,GPCP,GPCP_l,GPCP_l2,GPCP_g,GPCP_g_l,GPCP_g_fl,...,ccode_551_time,ccode_552_time,ccode_553_time,ccode_560_time,ccode_565_time,ccode_570_time,ccode_571_time,ccode_572_time,ccode_580_time,ccode_625_time
0,540.0,1981-01-01,Angola,AGO,839.215759,911.847290,1021.776855,-0.079653,-0.107587,0.155680,...,0,0,0,0,0,0,0,0,0,0
1,540.0,1982-01-01,Angola,AGO,969.864563,839.215759,911.847290,0.155680,-0.079653,-0.034482,...,0,0,0,0,0,0,0,0,0,0
2,540.0,1983-01-01,Angola,AGO,936.421631,969.864563,839.215759,-0.034482,0.155680,0.059925,...,0,0,0,0,0,0,0,0,0,0
3,540.0,1984-01-01,Angola,AGO,992.536255,936.421631,969.864563,0.059925,-0.034482,-0.018277,...,0,0,0,0,0,0,0,0,0,0
4,540.0,1985-01-01,Angola,AGO,974.396118,992.536255,936.421631,-0.018277,0.059925,0.216019,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,552.0,1995-01-01,Zimbabwe,ZWE,493.448456,465.000092,680.722412,0.061179,-0.316902,0.513642,...,0,17,0,0,0,0,0,0,0,0
739,552.0,1996-01-01,Zimbabwe,ZWE,746.904236,493.448456,465.000092,0.513642,0.061179,0.044380,...,0,18,0,0,0,0,0,0,0,0
740,552.0,1997-01-01,Zimbabwe,ZWE,780.051941,746.904236,493.448456,0.044380,0.513642,-0.181482,...,0,19,0,0,0,0,0,0,0,0
741,552.0,1998-01-01,Zimbabwe,ZWE,638.486389,780.051941,746.904236,-0.181482,0.044380,0.094420,...,0,20,0,0,0,0,0,0,0,0


In [11]:
table1 = repdata.loc[:,["any_prio", "any_prio_on", "any_prio_off","war_prio", "war_prio_on", "war_prio_off",
                        "war_col", "war_inc", "war","GPCP", "GPCP_g", "GPCP_g_l","gdp_g", "gdp_g_l",
        "y_0", "polity2l", "polity2l_6", "ethfrac", "relfrac", "Oil", "lmtnest", "lpopl1", "tot_100_g"]]

table1 

,any_prio,any_prio_on,any_prio_off,war_prio,war_prio_on,war_prio_off,war_col,war_inc,war,GPCP,...,gdp_g_l,y_0,polity2l,polity2l_6,ethfrac,relfrac,Oil,lmtnest,lpopl1,tot_100_g
0,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,839.215759,...,0.019637,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,8.933400,NaN
1,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,969.864563,...,-0.037037,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,8.959697,NaN
2,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,936.421631,...,0.044615,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,8.985946,NaN
3,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,992.536255,...,-0.008837,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,9.012134,NaN
4,1.0,NaN,0.0,1.0,NaN,0.0,1.0,1.0,1.0,974.396118,...,0.026746,0.662,-7.0,0.0,0.783282,0.6122,1.0,2.370244,9.035987,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,493.448456,...,0.030296,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.285019,-0.006359
739,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,746.904236,...,-0.018779,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.306650,0.001000
740,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0.0,780.051941,...,0.078490,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.327418,0.095405
741,1.0,1.0,NaN,1.0,1.0,NaN,0.0,0.0,0.0,638.486389,...,0.008137,1.190,-6.0,0.0,0.543950,0.5098,0.0,1.360977,9.347324,-0.006019


In [12]:
summary_table = table1.describe().loc[["mean","std","count"]].T # transponer DataFrame 

# Ahora el nombre de las variables son indices de las filas

summary_table

,mean,std,count
any_prio,0.267833,0.443128,743.0
any_prio_on,0.068468,0.252776,555.0
any_prio_off,0.143617,0.351638,188.0
war_prio,0.166891,0.373129,743.0
war_prio_on,0.036800,0.188421,625.0
war_prio_off,0.144068,0.352656,118.0
war_col,0.169583,0.375518,743.0
war_inc,0.220994,0.415203,724.0
war,0.244953,0.430350,743.0
GPCP,1001.638367,501.701569,743.0


In [13]:
table1.columns

new_names = ["Civil conflict with ≥25 deaths: (PRIO/Uppsala)","Onset","Offset",
"Civil conflict with ≥1,000 deaths:PRIO/Uppsala","Onset","Offset","Collier and Hoeffler (2002)",  
"Doyle and Sambanis (2000)","Fearon and Laitin (2003)",
"Annual rainfall (mm), GPCP measure",
"Annual growth in rainfall, time t",
"Annual growth in rainfall, time t-1",
"Annual economic growth rate, time t",
"Annual economic growth rate, time t-1",
"Log(GDP per capita), 1979",
"Democracy level (Polity IV score, -10 to 10), time t-1",
"Democracy indicator (Polity IV score > 15),time t-1",
"Ethnolinguistic fractionalization (source:Atlas Marodov Mira)",
"Religious fractionalization (source: CIAFactbook)",
"Oil-exporting country (source: WDI)",
"Log(mountainous) (source: Fearon and Laitin 2003)",
"Log(national population), time t-1 (source: WDI)",
"Growth in terms of trade, time t (source:WDI)"]

# unión de listas bajo la estructura diccionario

dict( zip( table1.columns, new_names) )

{'any_prio': 'Civil conflict with ≥25 deaths: (PRIO/Uppsala)',
 'any_prio_on': 'Onset',
 'any_prio_off': 'Offset',
 'war_prio': 'Civil conflict with ≥1,000 deaths:PRIO/Uppsala',
 'war_prio_on': 'Onset',
 'war_prio_off': 'Offset',
 'war_col': 'Collier and Hoeffler (2002)',
 'war_inc': 'Doyle and Sambanis (2000)',
 'war': 'Fearon and Laitin (2003)',
 'GPCP': 'Annual rainfall (mm), GPCP measure',
 'GPCP_g': 'Annual growth in rainfall, time t',
 'GPCP_g_l': 'Annual growth in rainfall, time t-1',
 'gdp_g': 'Annual economic growth rate, time t',
 'gdp_g_l': 'Annual economic growth rate, time t-1',
 'y_0': 'Log(GDP per capita), 1979',
 'polity2l': 'Democracy level (Polity IV score, -10 to 10), time t-1',
 'polity2l_6': 'Democracy indicator (Polity IV score > 15),time t-1',
 'ethfrac': 'Ethnolinguistic fractionalization (source:Atlas Marodov Mira)',
 'relfrac': 'Religious fractionalization (source: CIAFactbook)',
 'Oil': 'Oil-exporting country (source: WDI)',
 'lmtnest': 'Log(mountainous) (sou

In [14]:
# Customize summary table 

index = dict( zip( table1.columns, new_names) )

#nombre de columnas

columns = {
    "mean": "Mean",
    "std": "Standard Deviation",
    "count": "Observations",
}

# Rename rows (indexes) and columns
summary_table.rename(index=index, columns=columns, inplace=True)

In [15]:
summary_table

,Mean,Standard Deviation,Observations
Civil conflict with ≥25 deaths: (PRIO/Uppsala),0.267833,0.443128,743.0
Onset,0.068468,0.252776,555.0
Offset,0.143617,0.351638,188.0
"Civil conflict with ≥1,000 deaths:PRIO/Uppsala",0.166891,0.373129,743.0
Onset,0.036800,0.188421,625.0
Offset,0.144068,0.352656,118.0
Collier and Hoeffler (2002),0.169583,0.375518,743.0
Doyle and Sambanis (2000),0.220994,0.415203,724.0
Fearon and Laitin (2003),0.244953,0.430350,743.0
"Annual rainfall (mm), GPCP measure",1001.638367,501.701569,743.0


In [16]:

#Columns format

summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2).format(subset="Observations", precision=0)


# Export the DataFrame to LaTeX
# \ permite esccribir código extenso en lineas diferentes

summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2)\
.format(subset="Observations", precision=0)\
.to_latex(
    "summary2.tex",
caption="Descriptive Statistics",
    column_format = "lccc"
) 

In [17]:
summary_table.style.format(subset="Mean", precision=2).format(subset="Standard Deviation", precision=2).format(subset="Observations", precision=0)


,Mean,Standard Deviation,Observations
Civil conflict with ≥25 deaths: (PRIO/Uppsala),0.27,0.44,743
Onset,0.07,0.25,555
Offset,0.14,0.35,188
"Civil conflict with ≥1,000 deaths:PRIO/Uppsala",0.17,0.37,743
Onset,0.04,0.19,625
Offset,0.14,0.35,118
Collier and Hoeffler (2002),0.17,0.38,743
Doyle and Sambanis (2000),0.22,0.42,724
Fearon and Laitin (2003),0.24,0.43,743
"Annual rainfall (mm), GPCP measure",1001.64,501.70,743


## First stage 

#### Linear models libraries

https://www.statsmodels.org/stable/index.html

https://github.com/vgreg/python-se/blob/master/Standard%20errors%20in%20Python.ipynb

In [18]:
# vector y

y = repdata['gdp_g']

# add constant

X = sm.add_constant(repdata.loc[:,["GPCP_g", "GPCP_g_l"]])

# usando vectores y matrices como inputs

ols_model = sm.OLS(y, X).fit()

# fit permite correr la regresión

print(ols_model.summary())

# Robust standar error

ols_model_rb = sm.OLS(y, X).fit().get_robustcov_results(cov_type = "HC1")
print(ols_model_rb.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     8.912
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           0.000150
Time:                        16:49:44   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0061      0.003     -2.375      0.0

In [19]:
#alternative robust standar error

ols_model_rb0 = sm.OLS(y, X).fit(cov_type = "HC0") # White robust se
ols_model_rb1 = sm.OLS(y, X).fit(cov_type = "HC1") # Huber-White robust se
ols_model_rb2 = sm.OLS(y, X).fit(cov_type = "HC2") # Eicker-Huber-White robust
ols_model_rb3 = sm.OLS(y, X).fit(cov_type = "HC3")

print(ols_model_rb3.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     8.425
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           0.000241
Time:                        16:49:44   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0061      0.003     -2.345      0.0

In [20]:
# Acceder a la información de la tabla

ols_model_rb.summary2()

ols_model_rb.summary2().tables[1]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
const,-0.006147,0.002617,-2.348553,0.019109,-0.011285,-0.001009
GPCP_g,0.055430,0.014180,3.908990,0.000101,0.027592,0.083268
GPCP_g_l,0.034058,0.011901,2.861785,0.004332,0.010694,0.057422


In [21]:
# Observamos atributos y métodos

dir(sm.OLS(y, X))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_kwargs',
 '_data_attr',
 '_df_model',
 '_df_resid',
 '_fit_collinear',
 '_fit_ridge',
 '_fit_zeros',
 '_formula_max_endog',
 '_get_init_kwds',
 '_handle_data',
 '_init_keys',
 '_kwargs_allowed',
 '_setup_score_hess',
 '_sqrt_lasso',
 'data',
 'df_model',
 'df_resid',
 'endog',
 'endog_names',
 'exog',
 'exog_names',
 'fit',
 'fit_regularized',
 'from_formula',
 'get_distribution',
 'hessian',
 'hessian_factor',
 'information',
 'initialize',
 'k_constant',
 'loglike',
 'nobs',
 'predict',
 'rank',
 'score',
 'weights',
 'wendog',
 'wexog',
 'whiten']

In [22]:
# Recordad métodos y atributos de estructura de Clase 

dir(sm.OLS(y, X))

# predict para ello uso la función predict 

ols_model.predict()

# acceso al atributo parámetros

ols_model.params

# acceso a los atributos R2 y R2  ajustado

ols_model.rsquared
ols_model.rsquared_adj

0.02088027937651482

In [23]:
control_formula = "gdp_g"+ " ~ "+ "GPCP_g + " + "GPCP_g_l"

ols_model = smf.ols(control_formula, data=repdata).fit()

print(ols_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     8.912
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           0.000150
Time:                        16:49:44   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0061      0.003     -2.375      0.0

In [24]:
# ´predict 

ols_model.predict(X)

# X puede ser otra base de datos 

0     -0.014226
1     -0.000230
2     -0.002756
3     -0.004000
4     -0.005119
         ...   
738   -0.013549
739    0.024408
740    0.013807
741   -0.014695
742   -0.007094
Length: 743, dtype: float64

### Using sklearn (modelos ML lineales)

https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py



In [25]:
# Usando la libreria de modelos de machine learning 

ols_model_skl = linear_model.LinearRegression().fit( X, y )

ols_model_skl.coef_ # acceso a coeficientes 

ols_model_skl.predict(X) # predict en formato array 
ols_model_skl.score(X,y) # R cuadrado


0.023519415945896127

In [26]:
dir(ols_model_skl)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_n_features',
 '_decision_function',
 '_estimator_type',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_preprocess_data',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_residues',
 '_set_intercept',
 '_validate_data',
 'coef_',
 'copy_X',
 'feature_names_in_',
 'fit',
 'fit_intercept',
 'get_params',
 'intercept_',
 'n_features_in_',
 'n_jobs',
 'normalize',
 'positive',
 'predict',
 'rank_',
 'score',
 'set_params',
 'singular_']

In [27]:
mean_squared_error( y, ols_model.predict(X))**0.5 # root mean square error 

0.06981466645337808

## Modelo1 

- OLS, sin efectos fijos o country-time trend
- Robust errores estandar (Huber robust)
- Los residuos estan clusterizados (agrupados) a nivel país
-  Using cluster and robust standar error


In [28]:
formula_model1 = "gdp_g ~ GPCP_g + GPCP_g_l"

ols_model1 = smf.ols(formula_model1, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']})  
# cluster terminos de perturbación and robust estandar error 
print(ols_model1.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.021
Method:                 Least Squares   F-statistic:                     6.672
Date:                Fri, 11 Nov 2022   Prob (F-statistic):            0.00316
Time:                        16:49:44   Log-Likelihood:                 923.53
No. Observations:                 743   AIC:                            -1841.
Df Residuals:                     740   BIC:                            -1827.
Df Model:                           2                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0061      0.002     -2.499      0.0

In [29]:
print(ols_model1.summary().tables[1])

                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0061      0.002     -2.499      0.012      -0.011      -0.001
GPCP_g         0.0554      0.016      3.400      0.001       0.023       0.087
GPCP_g_l       0.0341      0.013      2.578      0.010       0.008       0.060


In [30]:
print(mean_squared_error( y, ols_model1.predict())**0.5) # Root mean square error
ols_model1.rsquared  # R2 square 

rms_model1 = round(mean_squared_error( y, ols_model1.predict())**0.5, 2) # redondeo a dos decimales

0.06981466645337808


### Model2 OLS:

- No efectos fijos (country), Si country-time trends
- errores estandar robustas (Huber-White robust)
- termino de perturbación están clusterizados (agrupados) a nivel país
- Se añade variables de control

In [31]:
# Control variables 

control_vars = ["y_0", "polity2l", "ethfrac", "relfrac", "Oil", "lmtnest","lpopl1"]

# country fixed effect

index_columns = np.where( repdata.columns.str.contains('_time$'))[0] # posición de las variables que terminan en time

country_trend = repdata.columns[index_columns] # selecciono el nombre de esas variables en un lista

formula_model2 = "gdp_g ~ GPCP_g + GPCP_g_l + " + ' + '.join( control_vars ) +' + ' + ' + '.join( country_trend )

# ' + '.join( control_vars )  une con con el simbolo de suma a todos los elementos de la lista control_vars

In [32]:
formula_model2

'gdp_g ~ GPCP_g + GPCP_g_l + y_0 + polity2l + ethfrac + relfrac + Oil + lmtnest + lpopl1 + ccode_404_time + ccode_420_time + ccode_432_time + ccode_433_time + ccode_434_time + ccode_435_time + ccode_436_time + ccode_437_time + ccode_438_time + ccode_439_time + ccode_450_time + ccode_451_time + ccode_452_time + ccode_461_time + ccode_471_time + ccode_475_time + ccode_481_time + ccode_482_time + ccode_483_time + ccode_484_time + ccode_490_time + ccode_500_time + ccode_501_time + ccode_510_time + ccode_516_time + ccode_517_time + ccode_520_time + ccode_522_time + ccode_530_time + ccode_540_time + ccode_541_time + ccode_551_time + ccode_552_time + ccode_553_time + ccode_560_time + ccode_565_time + ccode_570_time + ccode_571_time + ccode_572_time + ccode_580_time + ccode_625_time'

In [33]:
ols_model2 = smf.ols(formula_model2, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']}) 

# cluster terminos de perturbación and robust estándar error ante heterocedasticidad

print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.858
Date:                Fri, 11 Nov 2022   Prob (F-statistic):             0.0812
Time:                        16:49:45   Log-Likelihood:                 945.89
No. Observations:                 743   AIC:                            -1790.
Df Residuals:                     692   BIC:                            -1555.
Df Model:                          50                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.0439      0.064      0.

In [33]:
rms_model2 = round(mean_squared_error( y, ols_model2.predict())**0.5,2)

In [34]:
rms_model2 

0.07

### Model3 OLS:

- Efectos fijos (country), Si country-time trends
- errores estandar robustas (Huber-White robust)
- termino de perturbación están clusterizados (agrupados) a nivel país

In [91]:
# formula model 

formula_model3 = "gdp_g ~ GPCP_g + GPCP_g_l + C(ccode) + " + ' + '.join( country_trend )

# Se añade ccode como una variables categórica C(ccode). Con ello, se incluirá una dummy por cada país. 
# Para evitar trampa de dummies, el modelo omitirá una dummy

formula_model3

'gdp_g ~ GPCP_g + GPCP_g_l + C(ccode) + ccode_404_time + ccode_420_time + ccode_432_time + ccode_433_time + ccode_434_time + ccode_435_time + ccode_436_time + ccode_437_time + ccode_438_time + ccode_439_time + ccode_450_time + ccode_451_time + ccode_452_time + ccode_461_time + ccode_471_time + ccode_475_time + ccode_481_time + ccode_482_time + ccode_483_time + ccode_484_time + ccode_490_time + ccode_500_time + ccode_501_time + ccode_510_time + ccode_516_time + ccode_517_time + ccode_520_time + ccode_522_time + ccode_530_time + ccode_540_time + ccode_541_time + ccode_551_time + ccode_552_time + ccode_553_time + ccode_560_time + ccode_565_time + ccode_570_time + ccode_571_time + ccode_572_time + ccode_580_time + ccode_625_time'

In [92]:
ols_model3 = smf.ols(formula_model3, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']}) 
print(ols_model3.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     5.068
Date:                Fri, 11 Nov 2022   Prob (F-statistic):             0.0109
Time:                        16:14:34   Log-Likelihood:                 967.51
No. Observations:                 743   AIC:                            -1767.
Df Residuals:                     659   BIC:                            -1380.
Df Model:                          83                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0965      0.00

In [37]:
rms_model3 = round(mean_squared_error( y, ols_model3.predict())**0.5,2)
rms_model3

0.07

### Model4 OLS:

- Efectos fijos (country), Si country-time trends
- errores estandar robustas (Huber-White robust)
- termino de perturbación están clusterizados (agrupados) a nivel país
- Se añade la tasa de crecimeinto de las lluvias del periodo siguiente (GPCP_g_fl)

In [93]:
# formula model 4

formula_model4 = "gdp_g ~ GPCP_g + GPCP_g_l + GPCP_g_fl + C(ccode) + " + ' + '.join( country_trend )

ols_model4 = smf.ols(formula_model4, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']}) 
print(ols_model4.summary())

                            OLS Regression Results                            
Dep. Variable:                  gdp_g   R-squared:                       0.133
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     3.233
Date:                Fri, 11 Nov 2022   Prob (F-statistic):             0.0322
Time:                        16:15:14   Log-Likelihood:                 967.52
No. Observations:                 743   AIC:                            -1765.
Df Residuals:                     658   BIC:                            -1373.
Df Model:                          84                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0964      0.00

In [39]:
rms_model4 = round(mean_squared_error( y, ols_model4.predict())**0.5, 2)
rms_model4

0.07

### Model5 OLS:

- Efectos fijos (country), Si country-time trends
- errores estandar robustas (Huber-White robust)
- termino de perturbación están clusterizados (agrupados) a nivel país
- Se añade la tasa de crecimeinto de los terminos de intercambio (tot_100_g)

In [94]:
repdata.shape

(743, 283)

In [95]:
# La variable de control tot_100_g (tasa de variación de los términos de intercambio) presenta missings

repdata[['GPCP_g','GPCP_g_l', 'tot_100_g']].isna().sum()

GPCP_g        0
GPCP_g_l      0
tot_100_g    82
dtype: int64

In [96]:
# drop missing values en la columna tot_100_g
repdata_na = repdata.dropna(subset=['tot_100_g'])

In [97]:
# contabilizando el total de no missing en tot_100_g por país y añadiendolo en una columna llamada total_no_missing

repdata_na['total_no_missing'] = repdata_na.groupby('ccode')['tot_100_g'].transform('size') # size contabiliza no missing

In [98]:
repdata_na[['ccode','total_no_missing']]

,ccode,total_no_missing
5,540.0,13
6,540.0,13
7,540.0,13
8,540.0,13
9,540.0,13
...,...,...
738,552.0,19
739,552.0,19
740,552.0,19
741,552.0,19


In [99]:
# Se observa que non hay missing

repdata_na[['GPCP_g','GPCP_g_l', 'tot_100_g']].isna().sum()

GPCP_g       0
GPCP_g_l     0
tot_100_g    0
dtype: int64

In [103]:
#repdata_na = repdata.dropna(axis = 0) # drop NA by rows

# formula model 5

formula_model5 = "gdp_g ~ GPCP_g + GPCP_g_l + tot_100_g + C(ccode) + " + ' + '.join( country_trend )
ols_model5 = smf.ols(formula_model5, data=repdata_na).fit(cov_type='cluster', cov_kwds={'groups': repdata_na['ccode']}) 
ols_model5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  gdp_g   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     66.02
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           1.05e-14
Time:                        16:16:12   Log-Likelihood:                 926.36
No. Observations:                 661   AIC:                            -1699.
Df Residuals:                     584   BIC:                            -1353.
Df Model:                          76                                         
Covariance Type:              cluster                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.0962      0.003     29.022      0.000       0.090       0.103
C(ccode)[T.420.0]    -0.1529      0.006    -26.638      0.000      -0.164      -0.142
C(ccode)[T.432.0]    -0.1056      0.004    -28.179      0.000      -0.113      -0.098
C(ccode)[T.433.0]    -0.1047      0.003    -37.676      0.000      -0.110      -0.099
C(ccode)[T.434.0]    -0.1267      0.003    -47.862      0.000      -0.132      -0.122
C(ccode)[T.435.0]    -0.1091      0.004    -27.298      0.000      -0.117      -0.101
C(ccode)[T.436.0]    -0.1418      0.005    -26.959      0.000      -0.152      -0.132
C(ccode)[T.437.0]    -0.1533      0.003    -45.243      0.000      -0.160      -0.147
C(ccode)[T.438.0]    -0.1080      0.005    -20.399      0.000      -0.118      -0.098
C(ccode)[T.439.0]    -0.0838      0.002    -55.188      0.000      -0.087      -0.081
C(ccode)[T.451.0]    -0.1006      0.003    -32.434      0.000      -0.107      -0.095
C(ccode)[T.452.0]    -0.1394      0.005    -25.508      0.000      -0.150      -0.129
C(ccode)[T.461.0]    -0.1331      0.003    -39.318      0.000      -0.140      -0.126
C(ccode)[T.471.0]    -0.0746      0.002    -38.045      0.000      -0.078      -0.071
C(ccode)[T.475.0]    -0.1545      0.006    -24.798      0.000      -0.167      -0.142
C(ccode)[T.481.0]    -0.1372      0.004    -30.854      0.000      -0.146      -0.128
C(ccode)[T.482.0]    -0.1398      0.002    -82.664      0.000      -0.143      -0.136
C(ccode)[T.483.0]    -0.1550      0.004    -39.012      0.000      -0.163      -0.147
C(ccode)[T.484.0]    -0.0209      0.002     -9.134      0.000      -0.025      -0.016
C(ccode)[T.490.0]    -0.0788      0.002    -38.976      0.000      -0.083      -0.075
C(ccode)[T.500.0]    -0.2384      0.001   -182.497      0.000      -0.241      -0.236
C(ccode)[T.501.0]    -0.1081      0.004    -26.102      0.000      -0.116      -0.100
C(ccode)[T.510.0]    -0.1117      0.003    -38.935      0.000      -0.117      -0.106
C(ccode)[T.516.0]    -0.0571      0.003    -19.004      0.000      -0.063      -0.051
C(ccode)[T.517.0]    -0.1052      0.002    -44.903      0.000      -0.110      -0.101
C(ccode)[T.530.0]    -0.1111      0.004    -30.340      0.000      -0.118      -0.104
C(ccode)[T.540.0]    -0.1612      0.002    -73.637      0.000      -0.166      -0.157
C(ccode)[T.541.0]    -0.1600      0.004    -43.615      0.000      -0.167      -0.153
C(ccode)[T.551.0]    -0.1336      0.002    -66.637      0.000      -0.138      -0.130
C(ccode)[T.552.0]    -0.0971      0.003    -35.565      0.000      -0.102      -0.092
C(ccode)[T.553.0]    -0.1228      0.004    -31.253      0.000      -0.130      -0.115
C(ccode)[T.560.0]    -0.1074      0.004    -27.693      0.000      -0.115      -0.100
C(ccode)[T.565.0]    -0.1806      0.012    -15.205      0.000      -0.204

In [104]:
formula_model5 

'gdp_g ~ GPCP_g + GPCP_g_l + tot_100_g + C(ccode) + ccode_404_time + ccode_420_time + ccode_432_time + ccode_433_time + ccode_434_time + ccode_435_time + ccode_436_time + ccode_437_time + ccode_438_time + ccode_439_time + ccode_450_time + ccode_451_time + ccode_452_time + ccode_461_time + ccode_471_time + ccode_475_time + ccode_481_time + ccode_482_time + ccode_483_time + ccode_484_time + ccode_490_time + ccode_500_time + ccode_501_time + ccode_510_time + ccode_516_time + ccode_517_time + ccode_520_time + ccode_522_time + ccode_530_time + ccode_540_time + ccode_541_time + ccode_551_time + ccode_552_time + ccode_553_time + ccode_560_time + ccode_565_time + ccode_570_time + ccode_571_time + ccode_572_time + ccode_580_time + ccode_625_time'

In [105]:
dir(ols_model5)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abat_diagonal',
 '_cache',
 '_data_attr',
 '_data_in_cache',
 '_get_robustcov_results',
 '_is_nested',
 '_use_t',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_params_default',
 'cov_type',
 'df_model',
 'df_resid',
 'df_resid_inference',
 'diagn',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_p

In [106]:
rms_model5 = round(np.mean(ols_model5.resid**2)**0.5, 2)
rms_model5

0.06

### Export Latex table

In [107]:
exog_vars = ['GPCP_g','GPCP_g_l','GPCP_g_fl','tot_100_g','y_0', 'polity2l', 'ethfrac', 'relfrac', 'Oil', 'lmtnest', 'lpopl1'] 

var_labels = ["Growth in rainfall, t", "Growth in rainfall, t-1" ,"Growth in rainfall, t+1","Growth in terms of trade, t"
              "Log(GDP per capita), 1979",
"Democracy (Polity IV), t-1",
"Ethnolinguistic fractionalization",
"Religious fractionalization",
"Oil-exporting country",
"Log(mountainous)",
"Log(national population), t-1"]

# unión de listas bajo la estructura diccionario

labels_vars = dict( zip( exog_vars , var_labels) )

In [112]:
pystout(models=[ols_model1,ols_model2,ols_model3,ols_model4,ols_model5], file='latex_table_1.tex',
       digits=3,
        exogvars= exog_vars,
        varlabels= labels_vars ,
        modstat = { "rsquared":"R\sym{2}", "nobs":"Observations"} ,
        addrows={'Country fixed effects':['no','no','yes','yes','yes'],
                'Country-specific time trends':['no','yes','yes','yes','yes'],
                'Root mean square error':[rms_model1,rms_model2,rms_model3,rms_model4,rms_model5]},
        stars={.1:'*',.05:'**',.01:'***'},title="Rainfall and Economic Growth (First-Stage)",
        mgroups={'Ordinary Least Saquare':[1,5]},
       addnotes=["Note.— Huber robust standard errors are in parentheses.",
                 "Regression disturbance terms are clustered at the country level.",
       "* Significantly different from zero at 90 percent confidence.",
        "** Significantly different from zero at 95 percent confidence.",
        "*** Significantly different from zero at 99 percent confidence."])

In [113]:
# Otra alternativa

latex_export = summary_col([ols_model1,ols_model2,ols_model3,ols_model4,ols_model5], stars=True, float_format='%0.3f',
                          model_names=['(1)','(2)','(3)','(4)','(5)'],
                          info_dict={
                              'N':lambda x: "{0:d}".format(int(x.nobs))
                                    },
                          regressor_order=['GPCP_g','GPCP_g_l','GPCP_g_fl','tot_100_g',"y_0", "polity2l", "ethfrac", "relfrac", "Oil", "lmtnest","lpopl1"],
                          drop_omitted=True)

latex_export.add_title('Rainfall and Economic Growth (First-Stage)') ## add title                      

In [53]:
print(latex_export.as_latex() )

\begin{table}
\caption{Rainfall and Economic Growth (First-Stage)}
\label{}
\begin{center}
\begin{tabular}{llllll}
\hline
               & (1)      & (2)      & (3)      & (4)      & (5)       \\
\hline
GPCP\_g        & 0.055*** & 0.053*** & 0.049*** & 0.049*** & 0.053***  \\
               & (0.016)  & (0.017)  & (0.017)  & (0.018)  & (0.018)   \\
GPCP\_g\_l     & 0.034*** & 0.032**  & 0.028**  & 0.028**  & 0.037**   \\
               & (0.013)  & (0.014)  & (0.014)  & (0.014)  & (0.015)   \\
GPCP\_g\_fl    &          &          &          & 0.001    &           \\
               &          &          &          & (0.019)  &           \\
tot\_100\_g    &          &          &          &          & -0.002    \\
               &          &          &          &          & (0.023)   \\
y\_0           &          & -0.011   &          &          &           \\
               &          & (0.007)  &          &          &           \\
polity2l       &          & 0.000    &          &        

### 2.0 TABL4: OLS VERSUS 2SLS 

#### Logit and Probit
- variables de control
- No fixed effects by country 
- No trend-effects by country and year
- Variable temporal (time_year). El coeficiente de esta variable no se muestra en la tabla de resultado (PDF)

In [54]:
formula = "any_prio ~ gdp_g + gdp_g_l + time_year + " + ' + '.join( control_vars )

In [55]:
#Logit

logit_model = logit(formula, data = repdata).fit(cov_type="HC1")

Optimization terminated successfully.
         Current function value: 0.512248
         Iterations 6


In [56]:
# Efectos fijos evaluado en la media de cada explicativa

logit_me = logit_model.get_margeff(at = "mean", method = "dydx")
print(logit_me.summary())

        Logit Marginal Effects       
Dep. Variable:               any_prio
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gdp_g         -0.3786      0.306     -1.239      0.215      -0.977       0.220
gdp_g_l       -0.1087      0.278     -0.391      0.695      -0.653       0.436
time_year      0.0026      0.004      0.724      0.469      -0.004       0.010
y_0           -0.0653      0.024     -2.678      0.007      -0.113      -0.018
polity2l       0.0007      0.003      0.209      0.835      -0.006       0.007
ethfrac        0.2171      0.085      2.566      0.010       0.051       0.383
relfrac       -0.2903      0.101     -2.879      0.004      -0.488      -0.093
Oil            0.0069      0.060      0.114      0.909      -0.112       0.125
lmtnest        0.0739      0.014      5.191      0.000    

In [57]:
dir(logit_me)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_reset',
 'conf_int',
 'count_idx',
 'dummy_idx',
 'get_margeff',
 'margeff',
 'margeff_cov',
 'margeff_options',
 'margeff_se',
 'pvalues',
 'results',
 'summary',
 'summary_frame',
 'tvalues']

In [58]:
logit_me.summary_frame()

,dy/dx,Std. Err.,z,Pr(>|z|),Conf. Int. Low,Cont. Int. Hi.
gdp_g,-0.378576,0.305571,-1.238912,2.153780e-01,-0.977484,0.220333
gdp_g_l,-0.108737,0.277795,-0.391427,6.954814e-01,-0.653205,0.435732
time_year,0.002601,0.003594,0.723672,4.692669e-01,-0.004443,0.009644
y_0,-0.065326,0.024397,-2.677646,7.414148e-03,-0.113143,-0.017509
polity2l,0.000678,0.003247,0.208635,8.347330e-01,-0.005687,0.007042
ethfrac,0.217132,0.084610,2.566281,1.027955e-02,0.051300,0.382964
relfrac,-0.290261,0.100803,-2.879485,3.983258e-03,-0.487832,-0.092691
Oil,0.006874,0.060434,0.113741,9.094431e-01,-0.111574,0.125322
lmtnest,0.073943,0.014243,5.191496,2.086115e-07,0.046027,0.101859
lpopl1,0.078046,0.017854,4.371229,1.235489e-05,0.043052,0.113040


In [59]:
#Probit

probit_model = probit(formula, data = repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']})
probit_me = probit_model.get_margeff(at = "mean", method = "dydx")
print(probit_me.summary())

Optimization terminated successfully.
         Current function value: 0.509985
         Iterations 6
       Probit Marginal Effects       
Dep. Variable:               any_prio
Method:                          dydx
At:                              mean
                dy/dx    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gdp_g         -0.3747      0.265     -1.413      0.158      -0.895       0.145
gdp_g_l       -0.1362      0.236     -0.577      0.564      -0.599       0.327
time_year      0.0030      0.006      0.463      0.644      -0.010       0.016
y_0           -0.0672      0.062     -1.091      0.275      -0.188       0.053
polity2l       0.0007      0.005      0.140      0.889      -0.010       0.011
ethfrac        0.2358      0.261      0.903      0.367      -0.276       0.748
relfrac       -0.2898      0.260     -1.116      0.264      -0.799       0.219
Oil            0.0162      0.208   

In [60]:
# Export a latex 

print(probit_me.summary().as_latex())

\begin{center}
\begin{tabular}{lc}
\toprule
\textbf{Dep. Variable:} &   any\_prio     \\
\textbf{Method:}        &      dydx       \\
\textbf{At:}            &      mean       \\
\bottomrule
\end{tabular}
\begin{tabular}{ccccccc}
     \textbf{}      & \textbf{dy/dx} & \textbf{std err} & \textbf{z} & \textbf{P$> |$z$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\midrule
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
\textbf{gdp\_g}     &      -0.3747   &        0.265     &    -1.413  &         0.158        &       -0.895    &        0.145     \\
\textbf{gdp\_g\_l}  &      -0.1362   &        0.236     &    -0.577  &         0.564        &       -0.599    &        0.327     \\
\textbf{time\_year} &       0.0030   &        0.006     &     0.463  &         0.644        &       -0.010    &        0.016     \\
\textbf{y\_0}       &      -0.0672   &        0.062     &    -1.091  &         0.275        &       -0.188    &        0.053     \\
\textbf{polity2l}   &       0.0007   &        0.005

#### Model 2 (OLS)

- variables de control
- No fixed effects by country 
- No trend-effects by country and year
- Variable temporal (time_year). El coeficiente de esta variable no se muestra en la tabla de resultado (PDF)

In [61]:
# formula model 2

formula_model2 = "any_prio ~ gdp_g + gdp_g_l + time_year + " + ' + '.join( control_vars )

ols_model2 = smf.ols(formula_model2, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']})

# cov_type='cluster', cov_kwds={'groups': repdata['ccode']} # cluster terminos de perturbación and robust
print(ols_model2.summary())

                            OLS Regression Results                            
Dep. Variable:               any_prio   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.114
Method:                 Least Squares   F-statistic:                     2.967
Date:                Fri, 11 Nov 2022   Prob (F-statistic):            0.00693
Time:                        15:47:34   Log-Likelihood:                -399.18
No. Observations:                 743   AIC:                             820.4
Df Residuals:                     732   BIC:                             871.1
Df Model:                          10                                         
Covariance Type:              cluster                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4696      0.340     -1.383      0.1

In [74]:
# rmse 

y = repdata['any_prio'] # seleccionamos nuestro nuevo outcome (variable de interés)
rms_model2 = round(mean_squared_error( y, ols_model2.predict())**0.5,2)
rms_model2

0.41

#### Model 3 (OLS)

- variables de control
- No fixed effects by country 
- Si trend-effects by country and year

In [63]:
# formula model 3 (OLS)

formula_model3 = "any_prio ~ gdp_g + gdp_g_l + " + ' + '.join( control_vars ) + " + " + ' + '.join( country_trend )

ols_model3 = smf.ols(formula_model3, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']})  
# cluster terminos de perturbación
print(ols_model3.summary())

rms_model3 = round(mean_squared_error( y, ols_model3.predict())**0.5,2)
print("RMSE es :", rms_model3)

                            OLS Regression Results                            
Dep. Variable:               any_prio   R-squared:                       0.530
Model:                            OLS   Adj. R-squared:                  0.496
Method:                 Least Squares   F-statistic:                     7.264
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           2.08e-06
Time:                        15:47:34   Log-Likelihood:                -168.64
No. Observations:                 743   AIC:                             439.3
Df Residuals:                     692   BIC:                             674.4
Df Model:                          50                                         
Covariance Type:              cluster                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.8759      0.636     -2.

#### Model 4 (OLS)

- No variables de control
- Si fixed effects by country 
- Si trend-effects by country and year

In [64]:
# formula model 4 (OLS)

formula_model4 = "any_prio ~ gdp_g + gdp_g_l + C(ccode) + " + ' + '.join( country_trend )

ols_model4 = smf.ols(formula_model4, data=repdata).fit(cov_type='cluster', cov_kwds={'groups': repdata['ccode']})

# cluster terminos de perturbación

print(ols_model4.summary())


rms_model4 = round(mean_squared_error( y, ols_model4.predict())**0.5, 2)
print("RMSE es :", rms_model4)

                            OLS Regression Results                            
Dep. Variable:               any_prio   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.670
Method:                 Least Squares   F-statistic:                     17.17
Date:                Fri, 11 Nov 2022   Prob (F-statistic):           4.13e-06
Time:                        15:47:34   Log-Likelihood:                 6.6457
No. Observations:                 743   AIC:                             154.7
Df Residuals:                     659   BIC:                             542.0
Df Model:                          83                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.2361      0.02

#### Model 5 (IV-2SLS)

- Si variables de control
- No fixed effects by country 
- Si trend-effects by country and year

- Documentation

https://bashtage.github.io/linearmodels/iv/iv/linearmodels.iv.model.IV2SLS.fit.html#linearmodels.iv.model.IV2SLS.fit

In [65]:
# formula model 5 (IV2SLS)

formula_model5 = "any_prio ~ 1 + [ gdp_g + gdp_g_l ~ GPCP_g + GPCP_g_l ] + " + ' + '.join( control_vars ) + " + " + ' + '.join( country_trend )
ols_model5 = IV2SLS.from_formula(formula_model5, data=repdata).fit(cov_type = "clustered", 
                                                                   debiased=True,clusters = repdata["ccode"]) 
print(ols_model5)

rms_model5 = round(mean_squared_error( y, ols_model5.predict())**0.5, 2)
print("RMSE es :", rms_model5)

# “cluster” - One-way cluster dependent inference. Heteroskedasticity robust
# Flag indicating whether to debiased the covariance estimator using a degree of freedom adjustment.
# Se debe añadir el intercepto de manera explícita

                          IV-2SLS Estimation Summary                          
Dep. Variable:               any_prio   R-squared:                      0.4012
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3579
No. Observations:                 743   F-statistic:                 4.223e+14
Date:                Fri, Nov 11 2022   P-value (F-stat)                0.0000
Time:                        15:47:34   Distribution:                F(50,692)
Cov. Estimator:             clustered                                         
                                                                              
                               Parameter Estimates                                
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
Intercept         -1.7449     0.6840    -2.5510     0.0110     -3.0878     -0.4019
Oil               -0.1039     0.2185

In [66]:
ols_model5

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:               any_prio   R-squared:                      0.4012
Estimator:                    IV-2SLS   Adj. R-squared:                 0.3579
No. Observations:                 743   F-statistic:                 4.223e+14
Date:                Fri, Nov 11 2022   P-value (F-stat)                0.0000
Time:                        15:47:34   Distribution:                F(50,692)
Cov. Estimator:             clustered                                         
                                                                              
                               Parameter Estimates                                
==================================================================================
                Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
----------------------------------------------------------------------------------
Intercept         -1.7449     0.6840    -2.5510     0.0110     -3.0878     -0.4019
Oil               -0.1039     0.2185    -0.4757     0.6345     -0.5329      0.3250
ccode_404_time     0.0161     0.0122     1.3241     0.1859     -0.0078      0.0400
ccode_420_time     0.0120     0.0105     1.1349     0.2568     -0.0087      0.0326
ccode_432_time    -0.0041     0.0143    -0.2875     0.7738     -0.0322      0.0240
ccode_433_time     0.0269     0.0161     1.6697     0.0954     -0.0047      0.0585
ccode_434_time    -0.0052     0.0102    -0.5112     0.6094     -0.0253      0.0149
ccode_435_time     0.0243     0.0186     1.3083     0.1912     -0.0122      0.0608
ccode_436_time     0.0087     0.0089     0.9716     0.3316     -0.0089      0.0262
ccode_437_time    -0.0321     0.0158    -2.0366     0.0421     -0.0631     -0.0012
ccode_438_time     0.0009     0.0113     0.0823     0.9344     -0.0213      0.0231
ccode_439_time    -0.0006     0.0112    -0.0529     0.9578     -0.0225      0.0213
ccode_450_time     0.0272     0.0156     1.7412     0.0821     -0.0035      0.0579
ccode_451_time     0.0237     0.0078     3.0229     0.0026      0.0083      0.0391
ccode_452_time    -0.0185     0.0146    -1.2710     0.2041     -0.0471      0.0101
ccode_461_time     0.0013     0.0067     0.1893     0.8499     -0.0119      0.0145
ccode_471_time    -0.0358     0.0127    -2.8229     0.0049     -0.0608     -0.0109
ccode_475_time    -0.0496     0.0240    -2.0704     0.0388     -0.0967     -0.0026
ccode_481_time     0.0033     0.0168     0.1980     0.8431     -0.0296      0.0363
ccode_482_time    -0.0149     0.0117    -1.2698     0.2046     -0.0379      0.0081
ccode_483_time     0.0334     0.0099     3.3618     0.0008      0.0139      0.0529
ccode_484_time     0.0210     0.0136     1.5514     0.1213     -0.0056      0.0477
ccode_490_time    -0.0169     0.0148    -1.1444     0.2529     -0.0459      0.0121
ccode_500_time     0.0242     0.0132     1.8330     0.0672     -0.0017      0.0501
ccode_501_time    -0.0446     0.0142    -3.1367     0.0018     -0.0725     -0.0167
ccode_510_time    -0.0460     0.0147    -3.1292     0.0018     -0.0749     -0.0172
ccode_516_time     0.0345     0.0120     2.8650     0.0043      0.0108      0.0581
ccode_517_time     0.0371     0.0117     3.1647     0.0016      0.0141      0.0602
ccode_520_time     0.1166     0.0342     3.4097     0.0007      0.0495      0.1837
ccode_522_time     0.0482     0.0252     1.9154     0.0559     -0.0012      0.0976
ccode_530_time     0.0012     0.0164     0.0716     0.9429     -0.0310      0.0334
ccode_540_time     0.0446     0.0164     2.7103     0.0069      0.0123      0.0769
ccode_541_time     0.0076     0.0133     0.5712     0.5681     -0.0185      0.0336
ccode_551_time    -0.0226     0.0085    -2.6475     0.0083     -0.0394     -0.0058
ccode_552_time    -0.0027     0.0113    -0.2388     0.8113     -0.0250      0.0195
ccode_553_time    -0.0200     0.0085    -2.3399     0.0196     -0.0367     -0.

In [67]:
dir(ols_model5)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cov',
 '_cov_config',
 '_cov_estimator',
 '_cov_type',
 '_datetime',
 '_debiased',
 '_df_model',
 '_endogeneity_setup',
 '_f_statistic',
 '_fitted',
 '_kappa',
 '_liml_kappa',
 '_method',
 '_original_index',
 '_out_of_sample',
 '_params',
 '_r2',
 '_repr_html_',
 '_resid',
 '_rss',
 '_s2',
 '_top_right',
 '_tss',
 '_update_extra_text',
 '_vars',
 '_wresid',
 'anderson_rubin',
 'basmann',
 'basmann_f',
 'conf_int',
 'cov',
 'cov_config',
 'cov_estimator',
 'cov_type',
 'debiased',
 'df_model',
 'df_resid',
 'durbin',
 'f_statistic',
 'first_stage',
 'fitted_values',
 'has_constant',
 'idiosyncratic',
 'kappa',
 'method',

In [68]:
ols_model5._r2

0.4012016111391006

In [69]:
ols_model5.first_stage

,gdp_g,gdp_g_l
R-squared,0.0806,0.0723
Partial R-squared,0.0219,0.0161
Shea's R-squared,0.0219,0.0161
Partial F-statistic,11.478,7.8691
P-value (Partial F-stat),0.0032,0.0196
Partial F-stat Distn,chi2(2),chi2(2)
==========================,============,===========
Intercept,0.0439,0.0511
,(0.6839),(0.7535)
Oil,0.0067,0.0233


#### Model 6 (IV-2SLS)

- No variables de control
- Si fixed effects by country 
- Si trend-effects by country and year


In [70]:
# formula model 6 (IV2SLS)

formula_model6 = "any_prio ~ 1 + [ gdp_g + gdp_g_l ~ GPCP_g + GPCP_g_l ] + C(ccode) + " + " + " + ' + '.join( country_trend )
ols_model6 = IV2SLS.from_formula(formula_model6, data=repdata).fit(cov_type = "clustered", 
                                                                   debiased=True,clusters = repdata["ccode"]) 
print(ols_model6)

rms_model6 = round(mean_squared_error( y, ols_model6.predict())**0.5, 2)
print("RMSE es :", rms_model6)


                          IV-2SLS Estimation Summary                          
Dep. Variable:               any_prio   R-squared:                      0.5348
Estimator:                    IV-2SLS   Adj. R-squared:                 0.4762
No. Observations:                 743   F-statistic:                 1.761e+18
Date:                Fri, Nov 11 2022   P-value (F-stat)                0.0000
Time:                        15:47:34   Distribution:                F(83,659)
Cov. Estimator:             clustered                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
Intercept             0.0327     0.1817     0.1799     0.8573     -0.3242      0.3896
C(ccode)[T.420.0]     0.

#### Model 7 (IV-2SLS)

- y: Dependent Variable:Civil Conflict ≥1,000 Deaths
- No variables de control
- Si fixed effects by country 
- Si trend-effects by country and year


In [71]:
# formula model 7 (IV2SLS)

formula_model7 = "war_prio ~ 1 + [ gdp_g + gdp_g_l ~ GPCP_g + GPCP_g_l ] + C(ccode) + " + " + " + ' + '.join( country_trend )
ols_model7 = IV2SLS.from_formula(formula_model7, data=repdata).fit(cov_type = "clustered", 
                                                                   debiased=True,clusters = repdata["ccode"]) 
print(ols_model7)

rms_model7 = round(mean_squared_error( repdata['war_prio'], ols_model7.predict())**0.5, 2)
print("RMSE es :", rms_model7)


                          IV-2SLS Estimation Summary                          
Dep. Variable:               war_prio   R-squared:                      0.6230
Estimator:                    IV-2SLS   Adj. R-squared:                 0.5755
No. Observations:                 743   F-statistic:                -4.013e+18
Date:                Fri, Nov 11 2022   P-value (F-stat)                1.0000
Time:                        15:47:35   Distribution:                F(83,659)
Cov. Estimator:             clustered                                         
                                                                              
                                 Parameter Estimates                                 
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
Intercept             0.0838     0.0992     0.8443     0.3988     -0.1111      0.2786
C(ccode)[T.420.0]    -0.

In [72]:
exog_vars = ['gdp_g','gdp_g_l','y_0', 'polity2l', 'ethfrac', 'relfrac', 'Oil', 'lmtnest', 'lpopl1'] 

var_labels = ["Economic growth rate, t", "Economic growth rate, t-1" , "Log(GDP per capita), 1979",
"Democracy (Polity IV), t-1",
"Ethnolinguistic fractionalization",
"Religious fractionalization",
"Oil-exporting country",
"Log(mountainous)",
"Log(national population), t-1"]

# unión de listas bajo la estructura diccionario

labels_vars = dict( zip( exog_vars , var_labels) )

In [110]:
pystout(models=[ols_model2,ols_model3,ols_model4,ols_model5,ols_model6,ols_model7], file='latex_table_2.tex',
       digits=3,
        exogvars= exog_vars,
        varlabels= labels_vars ,
        modstat = { "rsquared":"R\sym{2}", "nobs":"Observations"} ,
        addrows={'Country fixed effects':['no','no','yes','no','yes','yes'],
                'Country-specific time trends':['no','yes','yes','yes','yes','yes'],
                'Root mean square error':[rms_model2,rms_model3,rms_model4,rms_model5,rms_model6,rms_model7]},
        stars={.1:'*',.05:'**',.01:'***'},title="Economic Growth and Civil Conflict",
        mgroups={'Ordinary Least Saquare':[1,6]},
       addnotes=["Note.— Huber robust standard errors are in parentheses.",
                 "Regression disturbance terms are clustered at the country level.",
              "* Significantly different from zero at 90 percent confidence.",
        "** Significantly different from zero at 95 percent confidence.",
        "*** Significantly different from zero at 99 percent confidence."])

#### pystout

https://pypi.org/project/pystout/